<a href="https://colab.research.google.com/github/Capstone-RSD/.github/blob/main/road1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
# clone YOLOv5 repository
# mount Google drive and create a "Road" directory
%cd /workspace/
!pwd
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

[Errno 2] No such file or directory: '/workspace/'
/yolov5/yolov5/yolov5
/yolov5/yolov5/yolov5
Cloning into 'yolov5'...
remote: Enumerating objects: 14967, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 14967 (delta 13), reused 16 (delta 6), pack-reused 14936
Receiving objects: 100% (14967/14967), 13.95 MiB | 19.05 MiB/s, done.
Resolving deltas: 100% (10277/10277), done.
/yolov5/yolov5/yolov5/yolov5
HEAD is now at fbe67e4 Fix `OMP_NUM_THREADS=1` for macOS (#8624)


In [34]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch.cuda

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# Check whether model is using GPU
if torch.cuda.is_available():
    device = torch.device("cuda") 
    !nvidia-smi
else:
    device = torch.device("cpu")

print(f'Using {device} for inference')

Setup complete. Using torch 1.13.1 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3070', major=8, minor=6, total_memory=8191MB, multi_processor_count=46)
Thu Jan 12 06:05:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.12    Driver Version: 527.41       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   43C    P8    24W / 270W |    958MiB /  8192MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------

In [35]:
!pip install -q roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="roboflow-yolov5")

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=roboflow-yolov5


In [36]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="cWbrW60dsEnAget6vKGJ")
project = rf.workspace("wzhen-vt-edu").project("pavement-cracks-2wi3m")
dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to pavement-cracks-1 in yolov5pytorch:: 100%|██████████| 7840/7840 [00:00<00:00, 21085.93it/s]


In [37]:
!pwd
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

/yolov5/yolov5/yolov5/yolov5
names:
- alligator cracking
- edge cracking
- longitudinal cracking
- patching
- pothole
- rutting
- transverse cracking
nc: 7
train: pavement-cracks-1/train/images
val: pavement-cracks-1/valid/images


In [38]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])
    print(num_classes)

7


In [43]:
%cat /workspace/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [44]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [45]:
%%writetemplate /workspace/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [46]:
%cat /workspace/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [48]:
# train yolov5s on custom data for 100 epochs
# time its performance
%time
%cd /workspace/yolov5/
!python train.py --img 416 --batch 16 --epochs 100 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache CUDA_LAUNCH_BLOCKING=1

CPU times: user 0 ns, sys: 1e+03 ns, total: 1e+03 ns
Wall time: 2.86 µs
/workspace/yolov5
train: weights=, cfg=./models/custom_yolov5s.yaml, data=/yolov5/yolov5/yolov5/yolov5/pavement-cracks-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=CUDA_LAUNCH_BLOCKING=1, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (Docker image), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.10.8 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)

hyperparameters: lr0=0.01, lrf=0.01, mo

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason... 
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/workspace/yolov5/runs/train/yolov5s_results/results.png', width=1000)  # view results.png

In [ ]:
# first, display our ground truth data
print("GROUND TRUTH TRAINING DATA:")
Image(filename='/workspace/yolov5/runs/train/yolov5s_results/val_batch0_labels.jpg', width=900)

In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/workspace/yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900)

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/train/yolov5s_results/weights

In [ ]:
%cd /yolov5/
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source ./pavement-cracks-1/test/images

In [ ]:
import glob
from IPython.display import Image, display
c=0;
for imageName in glob.glob('/workspace/yolov5/runs/detect/exp3/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")
    c=c+1;
    if(c ==10):
      break;

In [ ]:
%cd /content/drive/MyDrive/Road/yolov5/
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source ./pavement-cracks-1/test/images --save-txt --nosave

In [ ]:
#initial the model
import os
import sys
from pathlib import Path

import torch
import torch.backends.cudnn as cudnn

ROOT = '/workspace/yolov5/'
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
#ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_coords, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync


device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
weights='runs/train/yolov5s_results/weights/best.pt'
data='pavement-cracks-1/data.yaml'
conf_thres=0.4
iou_thres=0.45
imgsz=[416,416]

torch.no_grad()
# Load model
device = select_device(device)
model = DetectMultiBackend(weights, device=device, dnn=False, data=data, fp16=False)
stride, names, pt = model.stride, model.names, model.pt
imgsz = check_img_size(imgsz, s=stride)  # check image size


In [ ]:
import numpy as np
source='pavement-cracks-1/test/images'
dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt)
bs = 1  # batch_size
vid_path, vid_writer = [None] * bs, [None] * bs

# Run inference
model.warmup(imgsz=(1 if pt else bs, 3, *imgsz))  # warmup
seen, windows, dt = 0, [], [0.0, 0.0, 0.0]
for path, im, im0s, vid_cap, s in dataset:
    t1 = time_sync()
    im = torch.from_numpy(im).to(device)
    im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
    im /= 255  # 0 - 255 to 0.0 - 1.0
    if len(im.shape) == 3:
        im = im[None]  # expand for batch dim
    t2 = time_sync()
    dt[0] += t2 - t1

    # Inference
    #visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
    pred = model(im, augment=None, visualize=False)
    t3 = time_sync()
    dt[1] += t3 - t2

    # NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres, None, False, max_det=1000)
    dt[2] += time_sync() - t3

    # Second-stage classifier (optional)
    # pred = utils.general.apply_classifier(pred, classifier_model, im, im0s)
    seen += 1
    p = path
    p = Path(p)  # to Path
    print(p)
    det=pred[0]
    if len(det):
        # Rescale boxes from img_size to im0 size
        pred2=scale_coords(im.shape[2:], det[:, :4], im0s.shape).round()
        print(det.shape)
        print('box: '+str(np.array(pred2.cpu())))
        print('class: '+str(det[:,-1].cpu()))
        print('confidence: '+str(np.array(det[:,-2].cpu())))
        #break;
            


In [ ]:
import numpy as np
from utils.augmentations import letterbox
imagePath="/content/drive/MyDrive/Road/yolov5/pavement-cracks-1/test/images/us14--38-_jpg.rf.7067e4519392d181489b82ca5f8586c4.jpg"
img0 = cv2.imread(imagePath)

# Padded resize
img = letterbox(img0, imgsz, stride, auto=pt)[0]

# Convert
img = img.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
img = np.ascontiguousarray(img)

bs = 1  # batch_size

# Run inference
model.warmup(imgsz=(1 if pt else bs, 3, *imgsz))  # warmup
seen, windows, dt = 0, [], [0.0, 0.0, 0.0]

im = torch.from_numpy(img).to(device).cuda(device)
im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
im /= 255  # 0 - 255 to 0.0 - 1.0
if len(im.shape) == 3:
    im = im[None]  # expand for batch dim
t2 = time_sync()
dt[0] += t2 - t1

# Inference
#visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
pred = model(im, augment=None, visualize=False)
t3 = time_sync()
dt[1] += t3 - t2

# NMS
pred = non_max_suppression(pred, conf_thres, iou_thres, None, False, max_det=1000)
dt[2] += time_sync() - t3

# Second-stage classifier (optional)
# pred = utils.general.apply_classifier(pred, classifier_model, im, im0s)
seen += 1
p = path
p = Path(p)  # to Path
print(p)
det=pred[0]
if len(det):
    # Rescale boxes from img_size to im0 size
    pred2=scale_coords(im.shape[2:], det[:, :4], im0s.shape).round()
    print(det.shape)
    print('box: '+str(np.array(pred2.cpu())))
    print('class: '+str(det[:,-1].cpu()))
    print('confidence: '+str(np.array(det[:,-2].cpu())))
    #break;
        


In [ ]:
# what 's the meaning of the numbers (answered)
# [ xmin , ymin , xmax, ymax ]
import cv2
import matplotlib.pyplot as plt 
image = cv2.imread('/workspace/yolov5/pavement-cracks-1/test/images/us14--38-_jpg.rf.7067e4519392d181489b82ca5f8586c4.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
cv2.rectangle(image, (146, 0), (263, 395), (255,0,0), 2)
cv2.rectangle(image, (282, 320), (416, 342), (255,0,0), 2)
cv2.rectangle(image, (0, 261), (126, 339), (255,0,0), 2)
cv2.rectangle(image, (0, 75), (119, 94), (255,0,0), 2)
cv2.rectangle(image, (277, 61), (414, 83), (255,0,0), 2)
plt.imshow(image)
plt.show()

In [ ]:
!pip install neo4j

In [ ]:
import py2neo
import numpy as np

# Connect to Neo4j database
graph = py2neo.Graph(uri="bolt://localhost:7687", user="neo4j", password="soccer123")

# Export the model's state_dict to a file
torch.save(model.state_dict(), "model.pt")

# Node creation in Neo4j to store model data
model_data = model.state_dict()
for key, value in model_data.items():
    
    # Conver PyTorch to numpy array
    np_array = value.numpy()
    
    # Creating a node to store data
    cypher = f"CREATE (n:ModelData {{ name: '{key}', data: {np_array.tolist()} }})"
    graph.run(cypher)

# Verifying data
cypher = "MATCH (n:ModelData) RETURN n"
result = graph.run(cypher)
for record in result:
    print(record)